## tfkeras でモデルの保存と復元を試す
- Reference
  - https://www.tensorflow.org/tutorials/keras/save_and_load?hl=ja

In [1]:
#!pip install -q h5py pyyaml 

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os

import tensorflow as tf
from tensorflow import keras

tf.__version__

'2.0.0'

In [3]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

print(train_images.shape)

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

print(train_images.shape)

(60000, 28, 28)
(1000, 784)


In [4]:
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation='softmax')
  ])

  model.compile(optimizer='adam', 
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

  return model


model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [7]:
checkpoint_path = "data/training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

In [8]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, 
                                                 save_weights_only=True,
                                                 verbose=1)

model = create_model()

model.fit(train_images,
          train_labels,
          epochs = 10, 
          validation_data = (test_images, test_labels),
          callbacks = [cp_callback])

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 672/1000 [===================>..........] - ETA: 0s - loss: 1.4153 - accuracy: 0.6057 
Epoch 00001: saving model to data/training_1/cp.ckpt
1000/1000 [==============================] - 1s 807us/sample - loss: 1.1869 - accuracy: 0.6660 - val_loss: 0.7677 - val_accuracy: 0.7490
Epoch 2/10
 608/1000 [=================>............] - ETA: 0s - loss: 0.4849 - accuracy: 0.8618
Epoch 00002: saving model to data/training_1/cp.ckpt
1000/1000 [==============================] - 0s 164us/sample - loss: 0.4405 - accuracy: 0.8770 - val_loss: 0.5247 - val_accuracy: 0.8330
Epoch 3/10
 672/1000 [===================>..........] - ETA: 0s - loss: 0.2819 - accuracy: 0.9315
Epoch 00003: saving model to data/training_1/cp.ckpt
1000/1000 [==============================] - 0s 155us/sample - loss: 0.2933 - accuracy: 0.9250 - val_loss: 0.4760 - val_accuracy: 0.8530
Epoch 4/10
 640/1000 [==================>...........] - ETA: 0s - loss: 0.2223 - accura

In [5]:
model = create_model()

loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

1000/1 - 0s - loss: 2.2835 - accuracy: 0.1330
Untrained model, accuracy: 13.30%


In [8]:
model.load_weights(checkpoint_path)

loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1 - 0s - loss: 0.4808 - accuracy: 0.8740
Restored model, accuracy: 87.40%


### オプション

In [9]:
checkpoint_path = "data/training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

In [10]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path,
    verbose=1,
    save_weights_only=True,
    period=5)

model = create_model()

model.fit(train_images,
          train_labels,
          #epochs = 50,
          epochs = 20,
          callbacks = [cp_callback],
          validation_data = (test_images,test_labels),
          verbose=0)


Epoch 00005: saving model to data/training_2/cp-0005.ckpt

Epoch 00010: saving model to data/training_2/cp-0010.ckpt

Epoch 00015: saving model to data/training_2/cp-0015.ckpt

Epoch 00020: saving model to data/training_2/cp-0020.ckpt


In [11]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'data/training_2/cp-0020.ckpt'

In [12]:
model = create_model()
model.load_weights(latest)

loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1 - 0s - loss: 0.4243 - accuracy: 0.8760
Restored model, accuracy: 87.60%


### 手動で重みを保存

In [13]:
model.save_weights('data/checkpoints/my_checkpoint')

model = create_model()
model.load_weights('data/checkpoints/my_checkpoint')

loss,acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1 - 0s - loss: 0.4243 - accuracy: 0.8760
Restored model, accuracy: 87.60%


### モデル全体の保存

In [14]:
model = create_model()

model.fit(train_images, train_labels, epochs=5)

model.save('data/my_model.h5')

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 0s 343us/sample - loss: 1.1280 - accuracy: 0.6930
Epoch 2/5
1000/1000 [==============================] - 0s 83us/sample - loss: 0.4256 - accuracy: 0.8770
Epoch 3/5
1000/1000 [==============================] - 0s 82us/sample - loss: 0.2817 - accuracy: 0.9230
Epoch 4/5
1000/1000 [==============================] - 0s 84us/sample - loss: 0.2051 - accuracy: 0.9550
Epoch 5/5
1000/1000 [==============================] - 0s 83us/sample - loss: 0.1506 - accuracy: 0.9650


In [16]:
new_model = keras.models.load_model('data/my_model.h5')
new_model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [17]:
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1 - 0s - loss: 0.4352 - accuracy: 0.8550
Restored model, accuracy: 85.50%


In [18]:
model = create_model()

model.fit(train_images, train_labels, epochs=5)

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 0s 348us/sample - loss: 1.1667 - accuracy: 0.6310
Epoch 2/5
1000/1000 [==============================] - 0s 83us/sample - loss: 0.4368 - accuracy: 0.8870
Epoch 3/5
1000/1000 [==============================] - 0s 89us/sample - loss: 0.3137 - accuracy: 0.9110
Epoch 4/5
1000/1000 [==============================] - 0s 93us/sample - loss: 0.2198 - accuracy: 0.9410
Epoch 5/5
1000/1000 [==============================] - 0s 84us/sample - loss: 0.1686 - accuracy: 0.9600


In [19]:
import time
saved_model_path = "data/saved_models/{}".format(int(time.time()))

tf.keras.experimental.export_saved_model(model, saved_model_path)
saved_model_path

Instructions for updating:
Please use `model.save(..., save_format="tf")` or `tf.keras.models.save_model(..., save_format="tf")`.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: None
INFO:tensorflow:Signatures INCLUDED in export for Train: ['train']
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: None
INFO

'data/saved_models/1592898415'

In [20]:
new_model = tf.keras.experimental.load_from_saved_model(saved_model_path)
new_model.summary()

Instructions for updating:
The experimental save and load functions have been  deprecated. Please switch to `tf.keras.models.load_model`.
Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [21]:
model.predict(test_images).shape

(1000, 10)

In [22]:
new_model.compile(optimizer=model.optimizer,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1 - 0s - loss: 0.4301 - accuracy: 0.8630
Restored model, accuracy: 86.30%
